In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import glob
import os

from addict import Dict

from vmodel.util import color as vcolor

In [ ]:
plt.rcParams['figure.figsize'] = [4, 3.5]
plt.rcParams['figure.autolayout'] = True

# Pointmass

In [ ]:
data_dir_point = '/home/fabian/vmodel_datasets/neighbor_selection/pointmass'

In [ ]:
paths_point = sorted(glob.glob(data_dir_point + '/**/merged_*.nc', recursive=True))
len(paths_point)

In [ ]:
# Read datasets
expdictpoint = Dict()
for path in paths_point:
    dirname = os.path.dirname(path)
    dist_str, exp = dirname.split(os.path.sep)[-2:]
    dist = int(dist_str.replace('dist', ''))
    ds = xr.open_dataset(path)
    ds['nndist_min'] = ds.nndist.min('agent')
    del ds['nndist']
    ds['nvisible_mean'] = ds.nvisible.mean('agent')
    del ds['nvisible']
    expdictpoint[dist][exp] = ds

In [ ]:
distdictpoint = Dict()
for dist in expdictpoint:
    ds = xr.concat(expdictpoint[dist].values(), 'exp')
    ds.coords['exp'] = list(expdictpoint[dist].keys())
    distdictpoint[dist] = ds

In [ ]:
# Concatenate datasets along distance
dspointexps = xr.concat(distdictpoint.values(), 'dist')
dspointexps.coords['dist'] = list(distdictpoint.keys())

In [ ]:
if 'agent' in dspointexps.coords:
    del dspointexps.coords['agent']

In [ ]:
dspoint = dspointexps.sel(exp='visual_voronoi')
if 'exp' in dspoint.coords:
    del dspoint.coords['exp']

In [ ]:
dssizepointall = dspoint.sel(dist=1, drop=True)
dsdenspointall = dspoint.sel(nagents=100, drop=True)

# Quadcopter

In [ ]:
data_dir_quad = '/home/fabian/vmodel_datasets/neighbor_selection/quadcopter'

In [ ]:
paths_quad = sorted(glob.glob(data_dir_quad + '/**/merged_*.nc', recursive=True))
len(paths_quad)

In [ ]:
# Read datasets
expdictquad = Dict()
for path in paths_quad:
    dirname = os.path.dirname(path)
    stuff = dirname.split(os.path.sep)[-1]
    ds = xr.open_dataset(path)
    ds['nndist_min'] = ds.nndist.min('agent')
    del ds['nndist']
    ds['nvisible_mean'] = ds.nvisible.mean('agent')
    del ds['nvisible']
    expdictquad[stuff] = ds


In [ ]:
dsquad = xr.concat(expdictquad.values(), 'stuff')
dsquad.coords['stuff'] = list(expdictquad.keys())

In [ ]:
if 'agent' in dsquad.coords:
    del dsquad.coords['agent']

In [ ]:
dssizequadall = dsquad.sel(stuff='size', dist=1.0, drop=True)
dsdendsquadall = dsquad.sel(stuff='density', nagents=100, drop=True)

In [ ]:
labels = {
    'point': 'point mass',
    'quad': 'quadcopter'
}

In [ ]:
colors = {
    'point': vcolor.pointmass,
    'quad': vcolor.quadcopter
}

In [ ]:
timeslice = slice(150, 200)

In [ ]:
dst = ds.isel(time=timeslice).mean('time')

In [ ]:
distylim = (0.6, 1.2)
ordylim = (0.65, 1.05)

# Size

In [ ]:
dssizeall = xr.concat([dssizepointall, dssizequadall], 'exp')
dssizeall.coords['exp'] = ['point', 'quad']

In [ ]:
dssize = dssizeall.isel(time=timeslice).mean('time')

In [ ]:
def plot_size(ax, da):
    for exp in da.exp.data:
        data = da.sel(exp=exp)
        xs, ys, yerrs = data.nagents, data.mean('run'), data.std('run')
        label, color = labels[exp], colors[exp]
        ax.errorbar(xs, ys, yerrs, fmt='-o', capsize=3, label=label, color=color)
    ax.set(xscale='log')
    ax.set(xticks=da.nagents, xticklabels=da.nagents.data)
    ax.set(xlabel=r'number of agents $N$')
    ax.grid()
    ax.legend()

### Distance vs. group size

In [ ]:
fig, ax = plt.subplots()
plot_size(ax, dssize.nndist_min)
# ax.set(ylabel=r'min. nearest neighbor distance $d^\mathrm{min}$ [$m$]')
ax.set(ylabel=r'min. distance $d^\mathrm{min}$ [$m$]')
ax.set(ylim=distylim)
fig.savefig(f'size_nndist_min.pdf')

### Order vs. group size

In [ ]:
fig, ax = plt.subplots()
plot_size(ax, dssize.order)
ax.set(ylabel=r'avg. order $\phi^\mathrm{order}$')
ax.set(ylim=ordylim)
fig.savefig(f'size_order.pdf')

# Density

In [ ]:
dsdensall = xr.concat([dsdenspointall, dsdendsquadall], 'exp')
dsdensall.coords['exp'] = ['point', 'quad']

In [ ]:
dsdens = dsdensall.isel(time=timeslice).mean('time')

In [ ]:
def plot_density(ax, da):
    for exp in da.exp.data:
        data = da.sel(exp=exp)
        xs, ys, yerrs = data.dist, data.mean('run'), data.std('run')
        label, color = labels[exp], colors[exp]
        ax.errorbar(xs, ys, yerrs, fmt='-o', capsize=3, label=label, color=color)
    ax.set(xticks=da.dist, xticklabels=da.dist.data)
    ax.set(xlabel=r'reference distance $d^\mathrm{ref}$ [$m$]')
    ax.grid()
    ax.legend()

In [ ]:
dsdens['nndist_norm'] = dsdens.nndist_min / dsdens.dist

### Distance vs. density

In [ ]:
fig, ax = plt.subplots()
plot_density(ax, dsdens.nndist_norm)
ax.set(ylabel=r'norm. min. distance $d^\mathrm{norm}$')
ax.set(ylim=distylim)
fig.savefig(f'density_nndist_norm.pdf', bbox_inches='tight')
pass

### Order vs. density

In [ ]:
fig, ax = plt.subplots()
plot_density(ax, dsdens.order)
ax.set(ylabel=r'avg. order $\phi^{\mathrm{order}}$')
ax.set(ylim=ordylim)
fig.savefig(f'density_order.pdf', bbox_inches='tight')
pass